# Free Induction Decay

First of all, let's use the KomaMRI package and define the default scanner.

In [ ]:
using KomaMRI, Suppressor
sys = Scanner(); # default hardware definition

The free induction decay is the simplest observable NMR signal.
This signal is the one that follows a single tipping RF pulse.
To recreate this experiment, we will need to define a `Sequence` with 2 blocks.

The first block containing an RF pulse with a flip-angle of 90 deg,

In [ ]:
ampRF = 2e-6                        # 2 uT RF amplitude
durRF = π / 2 / (2π * γ * ampRF)    # required duration for a 90 deg RF pulse
exc = RF(ampRF,durRF);

and the second block containing the ADC.

In [ ]:
nADC = 8192         # number of acquisition samples
durADC = 250e-3     # duration of the acquisition
delay =  1e-3       # small delay
acq = ADC(nADC, durADC, delay);

Finally, we concatenate the sequence blocks to create
the final sequence.

In [ ]:
seq = Sequence()  # empty sequence
seq += exc        # adding RF-only block
seq += acq        # adding ADC-only block
p1 = plot_seq(seq; slider=false, height=300)

Now, we will define a `Phantom` with a single spin at $x=0$
with $T_1=1000\,\mathrm{ms}$ and $T_2=100\,\mathrm{ms}$.

In [ ]:
obj = Phantom(x=[0.], T1=[1000e-3], T2=[100e-3]);

Finally, to simulate we will need to use the function `simulate`
(note how the use of the `@suppress` macro prevents internal messages from being printed by the function):

In [ ]:
raw = @suppress simulate(obj, seq, sys);

To plot the results we will need to use the `plot_signal` function

In [ ]:
p2 = plot_signal(raw; slider=false, height=300)

Nice!, we can see that $S(t)$ follows an
exponential decay $\exp(-t/T_2)$ as expected.

For a little bit of spiciness, let's add **off-resonance** to our example.
We will use $\Delta f=-100\,\mathrm{Hz}$.
For this, we will need to add a definition for `Δw` in our `Phantom`

In [ ]:
obj = Phantom(x=[0.], T1=[1000e-3], T2=[100e-3], Δw=[-2π*100])# and simulate again.

raw = @suppress simulate(obj, seq, sys)
p3 = plot_signal(raw; slider=false, height=300)

The signal now follows an exponential of the
form $\exp(-t/T_2)\cdot\exp(-i\Delta\omega t)$.
The addition of $\exp(-i\Delta\omega t)$ to the signal
will generate a shift in the image space (Fourier shifting property).
This effect will be better visualized and explained in later examples.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*